In [ ]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/mw/work/

In [ ]:
# 查看当前kernel下的package
!pip list --format=columns

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [21]:
import pandas as pd
import gc
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix
%matplotlib inline
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
import lightgbm as lgb
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold,StratifiedKFold

In [59]:
# df_train = pd.read_csv('model_sample.csv')
df_train = pd.read_csv('features.csv',index_col =0)
df_train.reset_index()

df_train.head()

,user_id,y,x_001,x_002,x_003,x_004,x_005,x_006,x_007,x_008,...,x_184/x_179,x_185/x_180,x_189/x_188,x_191/x_190,x_193/x_192,x_195/x_194,x_197/x_196,x_199/x_198,x_196/x_188,x_192/x_188
0,A00002,0,0.0,32.0,0,0,0,0,0,0,...,0.25000,1000.000000,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
1,A00005,0,0.0,29.0,0,0,0,0,0,0,...,-999.00000,-999.000000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,A00006,0,0.0,31.0,0,0,0,0,0,0,...,0.62069,1606.939091,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0
3,A00008,0,0.0,22.0,0,0,0,0,0,0,...,0.00000,1570.506667,1.0,1.0,1.0,1.0,1.0,1.0,1.5,1.5
4,A00009,0,0.0,31.0,0,0,0,0,0,0,...,-999.00000,-999.000000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [26]:
no_features = ['y','user_id']
seed =2021
features = [col for col in df_train.columns if col not in no_features]

In [27]:
ycol = 'y'
feature_names = features
model = xgb.XGBClassifier(max_depth=6,
                          learning_rate=0.05,
                          n_estimators=10000,
                          subsample=0.8,
                          reg_alpha=10,
                          reg_lambda=12,
#                           tree_method='gpu_hist',
                        n_jobs=-1,
                          random_state=seed)


df_oof = []
df_importance_list = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for fold_id, (trn_idx, val_idx) in enumerate(
        kfold.split(df_train[feature_names],df_train[ycol])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]

    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(
        fold_id + 1))
    # print(df_oof.loc[val_idx]['prob'])
    xgb_model = model.fit(X_train,
                          Y_train,
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                          verbose=100,
                          eval_metric='auc',
                          early_stopping_rounds=50)

    pred_val = xgb_model.predict_proba(
        X_val)[:, 1]
   
    df_oof = df_train.iloc[val_idx][[
        'user_id', ycol]].copy()
    df_oof['pred'] = pred_val
    df_oof.append(df_oof)
    

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': xgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del xgb_model,  X_train, Y_train, X_val, Y_val
    gc.collect()


Fold_1 Training ================================

[0]	validation_0-auc:0.7602	validation_1-auc:0.737558
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
[100]	validation_0-auc:0.869961	validation_1-auc:0.793226
Stopping. Best iteration:
[112]	validation_0-auc:0.875786	validation_1-auc:0.794266


Fold_2 Training ================================

[0]	validation_0-auc:0.763628	validation_1-auc:0.737775
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
[100]	validation_0-auc:0.870144	validation_1-auc:0.787164
Stopping. Best iteration:
[133]	validation_0-auc:0.886576	validation_1-auc:0.788809


Fold_3 Training ================================

[0]	validation_0-auc:0.750548	validation_1-auc:0.728239
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early

In [44]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg(
    'mean').sort_values(ascending=False).reset_index()


In [45]:
df_importance.head()

,column,importance
0,x_122,0.069415
1,x_125/x_121,0.045721
2,x_126/x_127,0.022879
3,x_137,0.022614
4,x_139/x_137,0.022063


In [46]:
# df_oof.to_csv('xgb_oof',index=False)
df_oof.to_csv('xgb_oof_feature_engineering.csv',index = False)

In [47]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
Threshold = 0.36

In [48]:
recall_score(df_oof['y'], df_oof['pred'] >Threshold,average='macro')


0.6766336701471937

In [49]:
accuracy_score(df_oof['y'], df_oof['pred'] >Threshold)

0.7974568574023615

In [50]:
f1_score(df_oof['y'], df_oof['pred'] >Threshold,average='macro')


0.6766336701471937

In [51]:
precision_score(df_oof['y'], df_oof['pred'] >Threshold,average='macro')

0.6766336701471937

In [52]:
def get_roc_auc(y_test,y_score):
    # y_score =clf.predict_proba(test_vecs)[:,1]
    auc_s = roc_auc_score(y_test,y_score)   
    # print(y_predict)
    # print(y_score)
#     print(cm)
    print(auc_s)
    # Compute ROC curve and ROC area for each class
    fpr,tpr,threshold = roc_curve(y_test, y_score) ###计算真正率和假正率
    roc_auc = auc(fpr,tpr) ###计算auc的值
    
    plt.figure()
    lw = 2
    plt.figure(figsize=(10,10))
    plt.plot(fpr, tpr, color='darkorange',
            lw=lw, label='ROC curve (area = %0.2f)' % roc_auc) ###假正率为横坐标，真正率为纵坐标做曲线
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [53]:
get_roc_auc(df_oof['y'], df_oof['pred'])

0.7882037003866862


<Figure size 432x288 with 0 Axes>

<Figure size 720x720 with 1 Axes>

In [54]:


from sklearn.metrics import precision_recall_curve
# from sklearn.utils.fixes import signature
plt.figure("P-R Curve")
plt.title('Precision/Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
precision, recall, thresholds = precision_recall_curve(df_oof['y'], df_oof['pred'])
#print(precision)
#print(recall)
#print(thresholds)
plt.plot(recall,precision)
plt.show()

<Figure size 432x288 with 1 Axes>

In [55]:
fpr,tpr,thresholds= roc_curve(df_oof['y'], df_oof['pred'])
print ('KS:',max(tpr-fpr))

KS: 0.4423105290330738


In [56]:
def get_confusion_matrix(test_y,pred):
    f,ax=plt.subplots()

    C2= confusion_matrix(test_y, pred, labels=[0, 1])
    print(C2) #打印出来看看
    sns.set(font_scale=1.5)
    sns.heatmap(C2,annot=True,ax=ax, fmt='.20g',cmap="YlGnBu") #画热力图

    ax.set_title('confusion matrix') #标题
    ax.set_xlabel('predict') #x轴
    ax.set_ylabel('true') #y轴

In [57]:
get_confusion_matrix(df_oof['y'], df_oof['pred'] > 0.364)

[[1557  217]
 [ 228  200]]


<Figure size 432x288 with 2 Axes>

In [58]:
max_f1 = 0
Threshold = 0
for i in range(200,400):
    f1 = f1_score(df_oof['y'], df_oof['pred'] >i * 1e-3,average='macro')
    # f1 = f1_score(df_oof['y'].astype('int64'), (df_oof['pred'] >=i * 1e-3).astype('int64'),average='weighted')
    if max_f1 < f1:
        max_f1 = f1
        Threshold =  i

max_f1 ,Threshold * 1e-3

(0.676767192474603, 0.358)

In [43]:
df_oof.head()

,user_id,y,pred
2,A00006,0,0.403242
7,A00017,1,0.389520
30,A00064,0,0.055312
38,A00081,0,0.406562
41,A00086,0,0.043656
